In [1]:
# Import the requests module and the display, Image and HTML functions from the IPython.display module 
# and optionally the pandas module.
import requests
import json
from IPython.display import display, Image, HTML
import pandas as pd

In [2]:
# Ask the user for a programming language that they want to use for their search query. 
# Make sure to show the user which languages are available (you can choose which languages are available). 
# Also do input validation so that the language is valid.

print("Welcome! With this tool you can search articles within github repos. You can search in the following langauges: ")
programming_languages = ["python","javascript","php","java","ruby"]

for language in programming_languages:
    print(f'- {language}')

language_input = input("\nPlease choose your programming language of choice (from the list): ").lower().strip()
while language_input not in programming_languages:
    print("Please choose a programming language from the list.")
    language_input = input("Programming language of choice: ").lower().strip()

#Ask the user for a search query. Do input sanitization on this query (like stripping and lowercasing).
searchquery_input = input("Please enter your search query: ").lower().strip()

print("")
print(f"good! we are searching for '{searchquery_input}' in {language_input} projects.")

Welcome! With this tool you can search articles within github repos. You can search in the following langauges: 
- python
- javascript
- php
- java
- ruby

Please choose your programming language of choice (from the list): python
Please enter your search query: lists

good! we are searching for 'lists' in python projects.


In [3]:
# Use the Github search repositories API (https://developer.github.com/v3/search/#search-repositories) 
# to search for repositories with the search query, in the programming language the user has supplied and sorted by stars.
url = f"https://api.github.com/search/repositories?q={searchquery_input}+language:{language_input}&sort=stars&order=desc"
req = requests.get(url)

# Give feedback to the user that you are doing a HTTP call.
print(f'Doing a query for {url}')

# Check if the status code is 200, and if not, show an error message.
statuscode = req.status_code
if statuscode != 200:
    print(f'yikes. something is going wrong, error code {statuscode}')
else:
    print(f'we fine - statuscode {statuscode}.')
    
data = json.loads(req.text)

Doing a query for https://api.github.com/search/repositories?q=lists+language:python&sort=stars&order=desc
we fine - statuscode 200.


In [4]:
#Show the total_count of search results.
print(f"Total amount of search results: {data['total_count']}")

Total amount of search results: 3316


In [5]:
# Transform the search items to a new list with dictionaries containing the repository name, description, number of ‘stargazers’ and the username (‘login’) of the owner.
search_items = data["items"]
projects = []

for item in search_items:
    repo = { 
        "repo-name" : item["name"],
        "description" : item["description"],
        "stargazers" : item["stargazers_count"],
        "author" : item["owner"]["login"]
    }
    projects.append(repo)

In [6]:
df = pd.DataFrame(projects)
print("Here are the first 5 results:")
df.head(5)

Here are the first 5 results:


,author,description,repo-name,stargazers
0,kaxap,lists of most popular repositories for most fa...,arl,948
1,danielmagnussons,"orgmode is for keeping notes, maintaining TODO...",orgmode,706
2,hollobit,All About the GANs(Generative Adversarial Netw...,All-About-the-GAN,415
3,proycon,"PyNLPl, pronounced as 'pineapple', is a Python...",pynlpl,345
4,Pylons,A serialization/deserialization/validation lib...,colander,340


In [7]:
# Ask the user to select one of these five items. Do input validation so that the user chooses a valid option.
allowed_nrs = list(range(0,5))
nr = int(input("Please select an user from the list (0-4): ").strip())

while nr not in allowed_nrs:
    nr = int(input("Please select an user from the list (0-4): ").strip())

Please select an user from the list (0-4): 0


In [8]:
# Showing the user what option was chosen and getting the related user of that for the API.
chosen_user = df['author'][nr]
print(f"Perfect. You chose user {nr} ({chosen_user}).")

Perfect. You chose user 0 (kaxap).


In [9]:
# Use the Github user API to get details on this user. 
url = f"https://api.github.com/users/{chosen_user}"
req = requests.get(url)

# Give feedback to the user that you are doing a HTTP call.
print(f'Doing a query for {url}')

# Check if the status code is 200, and if not, show an error message.
statuscode = req.status_code
if statuscode != 200:
    print(f'yikes. something is going wrong, error code {statuscode}')
else:
    print(f'we fine - statuscode {statuscode}.')
    
user_info = json.loads(req.text)

Doing a query for https://api.github.com/users/kaxap
we fine - statuscode 200.


In [10]:
# From this API call extract the users name, number of public repos, bio, 
# the URL to the avatar image and the html_url with the link to the user’s page on Github.
name = user_info["name"]
public_repos = user_info["public_repos"]
bio = user_info["bio"]
avatar_url = user_info["avatar_url"]
html_url = user_info["html_url"]
if not bio:
    bio = "This user has no bio."
if not name:
    name = "This user has no public name available."
    
# Show the users name, public repos and bio. 
print(f'Name: {name}')
print(f'Total public repositories available: {public_repos}')
print(f'Bio: {bio}')

# If available, use the Image() and display() functions to show the users avatar image. 
if not avatar_url:
    print("This user has no avatar.")
else:
    print("")
    print("User Avatar:")
    img = Image(url = avatar_url)
    display(img)

# Also provide a clickable hyperlink to the users profile page on Github.
display(HTML(f'User profile on Github: <a href="{html_url}">Visit this user his/her Github Page!</a>'))

Name: This user has no public name available.
Total public repositories available: 34
Bio: This user has no bio.

User Avatar:
